In [2]:
import pandas as pd
import os
from IPython.display import display

# Define folder path (adjust if needed)
folder_path = '.'

# Get a list of all CSV files in the folder
file_list = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Separate files into baseline and frequency groups
baseline_files = [f for f in file_list if 'baseline' in f.lower()]
frequency_files = [f for f in file_list if 'frequency' in f.lower()]

# List of selected sensor IDs
sensor_ids = ['power', '12', '11','28', '26','27','29', 'tvoc', 'pm2p5', 'pm10p0', 'dewpoint']

# Function to process each file and compute mean & std values
def process_file(file_path):
    """Reads and processes a CSV file, computing mean and std for selected sensors."""
    data = pd.read_csv(file_path)

    # Ensure required columns exist
    if not {'sensor_id', 'value', 'time'}.issubset(data.columns):
        print(f"Error: Required columns missing in {file_path}")
        return None

    # Convert 'time' column to datetime and drop invalid values
    data['time'] = pd.to_datetime(data['time'], errors='coerce')
    data.dropna(subset=['time'], inplace=True)

    # Compute mean and std for selected sensors
    stats = data[data['sensor_id'].isin(sensor_ids)].groupby('sensor_id')['value'].agg(['mean', 'std']).round(2)
    stats['mean (std)'] = stats.apply(lambda x: f"{x['mean']} ({x['std']})", axis=1)
    return stats[['mean (std)']].reset_index()

# Process all files and store results
all_means = []
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    mean_values = process_file(file_path)
    if mean_values is not None:
        mean_values['file_name'] = file_name  # Track file source
        all_means.append(mean_values)

# Combine all mean values into a single DataFrame
if all_means:
    all_means_df = pd.concat(all_means).reset_index(drop=True)

    # Split into baseline and frequency groups
    baseline_df = all_means_df[all_means_df['file_name'].isin(baseline_files)]
    frequency_df = all_means_df[all_means_df['file_name'].isin(frequency_files)]

# Load and combine all sensor data
def load_combined_data(files):
    """Loads and combines sensor data from multiple files."""
    data_frames = [pd.read_csv(os.path.join(folder_path, f)) for f in files if os.path.exists(os.path.join(folder_path, f))]
    return pd.concat(data_frames, ignore_index=True) if data_frames else pd.DataFrame()

all_baseline_combined = load_combined_data(baseline_files)
all_frequency_combined = load_combined_data(frequency_files)

# Compute overall mean and std for baseline and frequency groups
if not all_baseline_combined.empty:
    overall_baseline_stats = all_baseline_combined[all_baseline_combined['sensor_id'].isin(sensor_ids)].groupby('sensor_id')['value'].agg(['mean', 'std']).round(2)
    overall_baseline_stats['mean (std)'] = overall_baseline_stats.apply(lambda x: f"{x['mean']} ({x['std']})", axis=1)
    print("\nOverall Baseline Stats:")
    display(overall_baseline_stats[['mean (std)']])

if not all_frequency_combined.empty:
    overall_frequency_stats = all_frequency_combined[all_frequency_combined['sensor_id'].isin(sensor_ids)].groupby('sensor_id')['value'].agg(['mean', 'std']).round(2)
    overall_frequency_stats['mean (std)'] = overall_frequency_stats.apply(lambda x: f"{x['mean']} ({x['std']})", axis=1)
    print("\nOverall Frequency Stats:")
    display(overall_frequency_stats[['mean (std)']])

# Extract sensor data recorded after 50 minutes
def extract_after_50_minutes(files):
    """Extracts sensor data recorded after 50 minutes from the first timestamp in each file."""
    filtered_entries = []
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        data = pd.read_csv(file_path)
        if not {'sensor_id', 'value', 'time'}.issubset(data.columns):
            continue

        # Convert time column to datetime
        data['time'] = pd.to_datetime(data['time'], errors='coerce')
        data.dropna(subset=['time'], inplace=True)
        
        # Determine threshold time (first timestamp + 50 minutes)
        if not data.empty:
            start_time = data['time'].min()
            threshold_time = start_time + pd.Timedelta(minutes=50)
            data_filtered = data[(data['sensor_id'].isin(sensor_ids)) & (data['time'] >= threshold_time)].copy()
            data_filtered['file_name'] = file_name
            filtered_entries.append(data_filtered)
    
    return pd.concat(filtered_entries, ignore_index=True) if filtered_entries else pd.DataFrame()

# Compute sensor data recorded after 50 minutes for both groups
baseline_after_50_df = extract_after_50_minutes(baseline_files)
frequency_after_50_df = extract_after_50_minutes(frequency_files)

# Compute mean and std of after 50 mins values
def compute_last_stats(df, group_name):
    """Computes mean and standard deviation of after 50 mins values."""
    if not df.empty:
        stats = df.groupby('sensor_id')['value'].agg(['mean', 'std']).round(2)
        stats['mean (std)'] = stats.apply(lambda x: f"{x['mean']} ({x['std']})", axis=1)
        print(f"\nMean and Standard Deviation of After 50 Minutes Data for {group_name}:")
        display(stats[['mean (std)']])

compute_last_stats(baseline_after_50_df, "Baseline")
compute_last_stats(frequency_after_50_df, "Frequency")



Overall Baseline Stats:


,mean (std)
sensor_id,
11,25.68 (1.38)
12,25.32 (1.47)
26,20.96 (0.23)
27,30.84 (8.37)
28,1018.53 (8.06)
29,638.83 (150.18)
dewpoint,3.9 (3.58)
pm10p0,1.22 (0.97)
pm2p5,1.08 (0.95)



Overall Frequency Stats:


,mean (std)
sensor_id,
11,17.03 (3.37)
12,16.55 (3.77)
26,20.97 (0.24)
27,31.25 (8.5)
28,1018.37 (8.09)
29,708.97 (219.46)
dewpoint,3.78 (3.81)
pm10p0,0.94 (0.46)
pm2p5,0.86 (0.46)



Mean and Standard Deviation of After 50 Minutes Data for Baseline:


,mean (std)
sensor_id,
11,25.75 (1.35)
12,25.39 (1.65)
26,20.97 (0.17)
27,31.12 (8.59)
28,1018.33 (8.12)
29,706.84 (162.92)
dewpoint,3.76 (3.71)
pm10p0,1.37 (1.22)
pm2p5,1.18 (1.18)



Mean and Standard Deviation of After 50 Minutes Data for Frequency:


,mean (std)
sensor_id,
11,20.53 (2.44)
12,20.07 (3.18)
26,20.96 (0.21)
27,31.97 (8.68)
28,1018.49 (8.12)
29,856.8 (240.63)
dewpoint,3.6 (4.01)
pm10p0,0.94 (0.63)
pm2p5,0.86 (0.63)
